In [1]:
%%time
import torch
import numpy as np
from scipy.optimize import minimize, Bounds
import cProfile, io, pstats
from pstats import SortKey
from botorch.models.gp_regression import SingleTaskGP
from botorch.acquisition.logei import qLogNoisyExpectedImprovement
from gpytorch import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_mll

torch.manual_seed(0)
np.random.seed(0)

torch.set_default_dtype(torch.double)

train_X = torch.rand(50, 2)
train_Y = torch.sin(train_X * 10).sum(dim=-1, keepdim=True)

model = SingleTaskGP(train_X=train_X, train_Y=train_Y)
model.posterior(train_X).mean  # set caches
acqf = qLogNoisyExpectedImprovement(model, train_X, cache_root=False)

bounds = Bounds(
    lb=np.zeros(128),
    ub=np.ones(128),
    keep_feasible=True,
)
x0 = np.random.random(128)

sortby = SortKey.CUMULATIVE


None

[KeOps] Warning : omp.h header is not in the path, disabling OpenMP. To fix this, you can set the environment
                  variable OMP_PATH to the location of the header before importing keopscore or pykeops,
                  e.g. using os.environ: import os; os.environ['OMP_PATH'] = '/path/to/omp/header'
[KeOps] Warning : Cuda libraries were not detected on the system or could not be loaded ; using cpu only mode
CPU times: user 3.59 s, sys: 1.62 s, total: 5.21 s
Wall time: 18.6 s


In [2]:
%%time

print("\n\n\n\n\n\n Optimize acqf \n\n")

def f_and_grad(x):
    X_tensor = torch.from_numpy(x.reshape((16, 4, 2))).contiguous().requires_grad_(True)
    loss = acqf(X_tensor).sum()
    grad = torch.autograd.grad(loss, X_tensor)[0].reshape(-1).numpy(force=True)
    return loss.item(), grad

with cProfile.Profile() as pr:
    res = minimize(
        fun=f_and_grad,
        x0=x0,
        method="L-BFGS-B",
        jac=True,
        bounds=bounds,
        options={"maxfun": 20},
    )

ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats(.1)
None







 Optimize acqf 




/opt/anaconda3/envs/botorch/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


         70166 function calls (67095 primitive calls) in 0.649 seconds

   Ordered by: cumulative time
   List reduced from 672 to 67 due to restriction <0.1>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.649    0.649 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_minimize.py:53(minimize)
        1    0.000    0.000    0.649    0.649 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_lbfgsb_py.py:290(_minimize_lbfgsb)
       46    0.001    0.000    0.647    0.014 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_optimize.py:71(_compute_if_needed)
       24    0.000    0.000    0.646    0.027 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:293(_update_fun)
       23    0.000    0.000    0.646    0.028 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:16(wr

In [3]:
%%time

print("\n\n\n\n\n\n 20 reference evaluations of f_and_grad \n\n")

X = np.random.random(128)

with cProfile.Profile() as pr:
    for _ in range(20):
        f_and_grad(X)

ps = pstats.Stats(pr).sort_stats(sortby)







 20 reference evaluations of f_and_grad 


CPU times: user 643 ms, sys: 512 ms, total: 1.16 s
Wall time: 393 ms


In [4]:
%%time

print("\n\n\n\n\n\n Optimize _get_samples_and_objectives \n\n")

def f_and_grad(x):
    X_tensor = torch.from_numpy(x.reshape((16, 4, 2))).contiguous().requires_grad_(True)
    loss = acqf._get_samples_and_objectives(X_tensor)[1].sum()
    grad = torch.autograd.grad(loss, X_tensor)[0].reshape(-1).numpy(force=True)
    return loss.item(), grad

with cProfile.Profile() as pr:
    res = minimize(
        fun=f_and_grad,
        x0=x0,
        method="L-BFGS-B",
        jac=True,
        bounds=bounds,
        options={"maxfun": 20},
    )

ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats(.1)
None







 Optimize _get_samples_and_objectives 


         60581 function calls (57872 primitive calls) in 0.253 seconds

   Ordered by: cumulative time
   List reduced from 435 to 44 due to restriction <0.1>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.253    0.253 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_minimize.py:53(minimize)
        1    0.000    0.000    0.253    0.253 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_lbfgsb_py.py:290(_minimize_lbfgsb)
       42    0.000    0.000    0.251    0.006 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_optimize.py:71(_compute_if_needed)
       22    0.000    0.000    0.251    0.011 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:293(_update_fun)
       21    0.000    0.000    0.251    0.012 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/sc

In [5]:
%%time

print("\n\n\n\n\n\n 20 reference evaluations of f_and_grad \n\n")

X = np.random.random(128)

with cProfile.Profile() as pr:
    for _ in range(20):
        f_and_grad(X)

ps = pstats.Stats(pr).sort_stats(sortby)








 20 reference evaluations of f_and_grad 


CPU times: user 656 ms, sys: 509 ms, total: 1.17 s
Wall time: 213 ms


In [16]:
%%time

print("\n\n\n\n\n\n Optimize model.posterior.rsample \n\n")

def f_and_grad(x):
    X_tensor = torch.from_numpy(x.reshape((16, 4, 2))).contiguous().requires_grad_(True)
    loss = model.posterior(X_tensor).rsample(torch.Size([128, 512])).sum()
    grad = torch.autograd.grad(loss, X_tensor)[0].reshape(-1).numpy(force=True)
    return loss.item(), grad

with cProfile.Profile() as pr:
    res = minimize(
        fun=f_and_grad,
        x0=x0,
        method="L-BFGS-B",
        jac=True,
        bounds=bounds,
        options={"maxfun": 20},
    )

ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats(.1)
None







 Optimize model.posterior.mean 


         56682 function calls (54120 primitive calls) in 1.741 seconds

   Ordered by: cumulative time
   List reduced from 409 to 41 due to restriction <0.1>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.741    1.741 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_minimize.py:53(minimize)
        1    0.000    0.000    1.741    1.741 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_lbfgsb_py.py:290(_minimize_lbfgsb)
       42    0.001    0.000    1.738    0.041 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_optimize.py:71(_compute_if_needed)
       22    0.000    0.000    1.738    0.079 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:293(_update_fun)
       21    0.000    0.000    1.738    0.083 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/opt

In [15]:
%%time

print("\n\n\n\n\n\n 20 reference evaluations of f_and_grad \n\n")

X = np.random.random(128)

with cProfile.Profile() as pr:
    for _ in range(20):
        f_and_grad(X)

ps = pstats.Stats(pr).sort_stats(sortby)








 20 reference evaluations of f_and_grad 


CPU times: user 481 ms, sys: 90.5 ms, total: 572 ms
Wall time: 89.4 ms


In [10]:
%%time

print("\n\n\n\n\n\n Optimize a simple matmul \n\n")

rand_tensor = torch.rand(128, 128, 128)

def f_and_grad(x):
    X_tensor = torch.from_numpy(x).contiguous().requires_grad_(True)
    diff = rand_tensor - X_tensor
    loss = (diff @ diff).sum()
    grad = torch.autograd.grad(loss, X_tensor)[0].reshape(-1).numpy(force=True)
    return loss.item(), grad

with cProfile.Profile() as pr:
    res = minimize(
        fun=f_and_grad,
        x0=x0,
        method="L-BFGS-B",
        jac=True,
        bounds=bounds,
        options={"maxfun": 20},
    )

ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats(.1)
None







 Optimize a simple root finding problem 


         2983 function calls in 0.337 seconds

   Ordered by: cumulative time
   List reduced from 136 to 14 due to restriction <0.1>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.337    0.337 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_minimize.py:53(minimize)
        1    0.000    0.000    0.336    0.336 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_lbfgsb_py.py:290(_minimize_lbfgsb)
       44    0.004    0.000    0.334    0.008 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_optimize.py:71(_compute_if_needed)
       23    0.000    0.000    0.334    0.015 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:293(_update_fun)
       22    0.000    0.000    0.334    0.015 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_different

In [11]:
%%time

print("\n\n\n\n\n\n 20 reference evaluations of f_and_grad \n\n")

X = np.random.random(128)

with cProfile.Profile() as pr:
    for _ in range(20):
        f_and_grad(X)

ps = pstats.Stats(pr).sort_stats(sortby)








 20 reference evaluations of f_and_grad 


CPU times: user 1.32 s, sys: 719 ms, total: 2.04 s
Wall time: 307 ms
